In [2]:
import os

import nibabel as nib
import numpy as np
import scipy.ndimage as ndi

datapath = "/media/lm/Samsung_T5/Uni/Medml/training/train"

files = os.listdir(datapath)
files = filter(lambda x: x.endswith("_orig.nii.gz"), files)
files = list(map(lambda x: x.replace("_orig.nii.gz", ""), files))
sorted(files)

volumes = []
for idx, name in enumerate(files):
    print(f"{idx} / {len(files)}: {name}")

    raw = nib.load(os.path.join(datapath, name + "_orig.nii.gz"))
    label = nib.load(os.path.join(datapath, name + "_masks.nii.gz"))

    raw_np = raw.get_fdata()
    label_np = label.get_fdata()

    raw_np = ndi.zoom(raw_np, (0.5, 0.5, 0.5), order=3)
    label_np = ndi.zoom(label_np, (0.5, 0.5, 0.5), order=0)
    label_np = label_np > 0.5

    perc = np.percentile(raw_np, 99)

    t = (raw_np > perc).astype(int)

    tt, num_labels = ndi.label(t)

    t2, num_labels_mask = ndi.label(label_np)
    for i in range(1, num_labels_mask+1):
        volume = np.sum(t2 == i)
        overlap = np.logical_and(t2 == i, t)
        overlap_size = (np.logical_and(t2 == i, t)).sum()

        x_idx, y_idx, z_idx = np.where(overlap)

        idxx = len(x_idx) // 2
        label = tt[x_idx[idxx], y_idx[idxx], z_idx[idxx]]

        artery_volume = np.sum(tt == label)

        volumes.append((overlap_size, artery_volume, volume))


# t = ndi.binary_dilation(t, iterations=1).astype(int)


0 / 76: A012
1 / 76: A100


KeyboardInterrupt: 

In [2]:
import pandas as pd
df = pd.DataFrame(volumes, columns=["overlap_size", "artery_volume", "volume"])
df.describe()

,overlap_size,artery_volume,volume
count,87.000000,87.000000,87.000000
mean,312.126437,8994.942529,352.137931
std,646.625081,2615.956201,713.402952
min,9.000000,2253.000000,12.000000
25%,48.500000,7632.000000,60.000000
50%,105.000000,9013.000000,122.000000
75%,293.000000,10453.000000,328.500000
max,4595.000000,16783.000000,5006.000000


In [41]:
import os
import shutil
### Split train test

data_path = "/media/lm/Samsung_T5/Uni/Medml/training"

train_split = 0.7
# Load files
files = os.listdir(data_path)
files = filter(lambda x: x.endswith("_orig.nii.gz"), files)
files = list(map(lambda x: x.replace("_orig.nii.gz", ""), files))
np.random.shuffle(files)

split = int(len(files) * train_split)
train = files[:split]
val = files[split:]

print(train, val)

def move(files_f, folder):
    folder = os.path.join(data_path, folder)
    if not os.path.exists(folder):
        os.makedirs(folder)

    for i in files_f:
        shutil.move(os.path.join(data_path, i + "_orig.nii.gz"), os.path.join(folder, i + "_orig.nii.gz"))
        shutil.move(os.path.join(data_path, i + "_masks.nii.gz"), os.path.join(folder, i + "_masks.nii.gz"))

move(train, "train")
move(val, "val")



['A012', 'A100', 'A056', 'A123', 'A057', 'A029', 'A041', 'A098', 'A067', 'A076', 'A091_R', 'A038_R', 'A130_L', 'PA5', 'A135', 'A083', 'A059_L', 'A086', 'A082', 'A050', 'A097', 'A071', 'A105_R', 'A014', 'PA6', 'A001', 'A017_L', 'A133', 'A040', 'A003', 'A074', 'A044', 'A084', 'A085', 'A066', 'A126', 'A064', 'A043', 'A079', 'A015', 'A027', 'A028', 'A062_L', 'A081', 'A070', 'A087', 'A103', 'A121', 'A138', 'A008', 'A051_R', 'A096_L', 'A112', 'A095', 'A080', 'A092', 'A130_R', 'A038_M', 'A096_R', 'A010', 'A026', 'A134', 'A078_L', 'A032', 'A038_L', 'A060', 'A119', 'A113', 'A108', 'A024', 'A094_R', 'A105_L', 'A046', 'A093', 'A089_R', 'A077'] ['A045', 'A129', 'A114', 'A118', 'A073', 'A006', 'A136', 'A047', 'A049', 'A025', 'A019', 'A023_R', 'A120', 'A127', 'A042', 'A115', 'A088', 'A137', 'A072', 'A016', 'A009', 'A068', 'A059_R', 'A018', 'A013', 'A031', 'A124', 'A035', 'A078_R', 'A021', 'A005', 'A033', 'A054']


In [10]:
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
case = "A003"
raw = nib.load(f"/media/lm/Samsung_T5/Uni/Medml/training/train/{case}_orig.nii.gz")
label = nib.load(f"/media/lm/Samsung_T5/Uni/Medml/training/train/{case}_masks.nii.gz")

raw_np = raw.get_fdata()
label_np = label.get_fdata()

raw_np = ndi.zoom(raw_np, (0.5, 0.5, 0.5), order=3)
label_np = ndi.zoom(label_np, (0.5, 0.5, 0.5), order=0)
label_np = label_np > 0.5

perc = np.percentile(raw_np, 99)

t = (raw_np > perc).astype(int)

tt, num_labels = ndi.label(t)
unique, counts = np.unique(tt, return_counts=True)

t = ndi.binary_closing(t, iterations=1)

keep_idx = unique[counts > 2000]
keep = None
for i in keep_idx:
    if i == 0:
        continue

    if keep is None:
        keep = tt == i
    else:
        keep = np.logical_or(keep, (tt == i))

remove_idx = np.logical_not(keep)

t[remove_idx] = 0

idx = np.argmax(t.sum(axis=(1, 2)))
print(idx)

#t = ndi.gaussian_filter(t, sigma=0.1, order=0)
# t = t > 0.5

# plt.figure(figsize=(20, 10))
# plt.subplot(1, 2, 1)
# plt.imshow(t[idx])
# plt.subplot(1, 2, 2)
# t = ndi.gaussian_filter(t, sigma=0.15, order=0)
# plt.imshow(t[idx])


nib.save(nib.Nifti1Image(t.astype(int), raw.affine), "./data/test_thres/art.nii.gz")
nib.save(nib.Nifti1Image(label_np.astype(int), raw.affine), "./data/test_thres/mask.nii.gz")

56


In [14]:
import h5py
import os

datapath = "/media/lm/Samsung_T5/Uni/Medml/training/train"

files = os.listdir(datapath)
files = filter(lambda x: x.endswith("_orig.nii.gz"), files)
files = list(map(lambda x: x.replace("_orig.nii.gz", ""), files))
sorted(files)

h5_save_folder = os.path.join(datapath, "h5")

if not os.path.exists(h5_save_folder):
    os.makedirs(h5_save_folder)

# PARAMS
zoom = False
volume_threshold = 30000 #2000
closing_thres = 3

volumes = []
for idx, name in enumerate(files):
    print(f"{idx} / {len(files)}: {name}")

    raw = nib.load(os.path.join(datapath, name + "_orig.nii.gz"))
    label = nib.load(os.path.join(datapath, name + "_masks.nii.gz"))

    raw_np = raw.get_fdata()
    label_np = label.get_fdata()

    if zoom:
        raw_np = ndi.zoom(raw_np, (0.5, 0.5, 0.5), order=3)
        label_np = ndi.zoom(label_np, (0.5, 0.5, 0.5), order=0)
        label_np = label_np > 0.5

    # Find Artery
    perc = np.percentile(raw_np, 99)

    t = (raw_np > perc).astype(int)

    tt, num_labels = ndi.label(t)
    unique, counts = np.unique(tt, return_counts=True)

    keep_idx = unique[counts > volume_threshold]
    keep = None
    for i in keep_idx:
        if i == 0:
            continue

        if keep is None:
            keep = tt == i
        else:
            keep = np.logical_or(keep, (tt == i))

    remove_idx = np.logical_not(keep)

    t[remove_idx] = 0

    t = ndi.binary_closing(t, iterations=closing_thres)

    overlap_mask = np.logical_and(label_np, t)

    # Normalize
    min_val = raw_np.min()
    max_val = raw_np.max()

    raw_np = (raw_np - min_val) / (max_val - min_val)

    with h5py.File(os.path.join(h5_save_folder, f"{name}.h5"), "w") as f:
        f.create_dataset("raw", data=raw_np)
        f.create_dataset("label", data=label_np)
        f.create_dataset("artery", data=t)
        f.create_dataset("overlap_mask", data=overlap_mask)

0 / 76: A012
1 / 76: A100
2 / 76: A056
3 / 76: A123
4 / 76: A057
5 / 76: A029
6 / 76: A041
7 / 76: A098
8 / 76: A067
9 / 76: A076
10 / 76: A091_R
11 / 76: A038_R
12 / 76: A130_L
13 / 76: PA5
14 / 76: A135
15 / 76: A083
16 / 76: A059_L
17 / 76: A086
18 / 76: A082
19 / 76: A050
20 / 76: A097
21 / 76: A071
22 / 76: A105_R
23 / 76: A014
24 / 76: PA6
25 / 76: A001
26 / 76: A017_L
27 / 76: A133
28 / 76: A040
29 / 76: A003
30 / 76: A074
31 / 76: A044
32 / 76: A084
33 / 76: A085
34 / 76: A066
35 / 76: A126
36 / 76: A064
37 / 76: A043
38 / 76: A079
39 / 76: A015
40 / 76: A027
41 / 76: A028
42 / 76: A062_L
43 / 76: A081
44 / 76: A070
45 / 76: A087
46 / 76: A103
47 / 76: A121
48 / 76: A138
49 / 76: A008
50 / 76: A051_R
51 / 76: A096_L
52 / 76: A112
53 / 76: A095
54 / 76: A080
55 / 76: A092
56 / 76: A130_R
57 / 76: A038_M
58 / 76: A096_R
59 / 76: A010
60 / 76: A026
61 / 76: A134
62 / 76: A078_L
63 / 76: A032
64 / 76: A038_L
65 / 76: A060
66 / 76: A119
67 / 76: A113
68 / 76: A108
69 / 76: A024
70 /

In [9]:
import h5py
import nibabel as nib
outpath = "./data/test_thres"

def h5_to_nii(p):
    with h5py.File(p, "r") as f:
        raw = f["raw"][:]
        mask = f["label"][:]
        artery = f["artery"][:]
        # overlap_mask = f["overlap_mask"][:]

        artery = ndi.binary_closing(artery, iterations=3)

        nib.save(nib.Nifti1Image(raw, np.eye(4)), f"{outpath}/raw.nii.gz")
        nib.save(nib.Nifti1Image(mask.astype(int), np.eye(4)), f"{outpath}/mask.nii.gz")
        nib.save(nib.Nifti1Image(artery.astype(float), np.eye(4)), f"{outpath}/artery.nii.gz")
        # nib.save(nib.Nifti1Image(overlap_mask.astype(int), np.eye(4)), f"{outpath}/overlap_mask.nii.gz")

h5_to_nii("/media/lm/Samsung_T5/Uni/Medml/training/train/h5/A001.h5")

In [34]:
# Calc stats
datapath = "/media/lm/Samsung_T5/Uni/Medml/training/train/h5_zoomed"

files = os.listdir(datapath)
sorted(files)

volumes = []
for idx, name in enumerate(files):
    if not name.endswith(".h5"):
        continue

    print(f"{idx} / {len(files)}: {name}")


    with h5py.File(os.path.join(datapath, name), "r") as f:
        raw = f["raw"][:]
        mask = f["label"][:]
        artery = f["artery"][:]

        artery_labels, num_artery_labels = ndi.label(artery)

        t2, num_labels_mask = ndi.label(mask)
        for i in range(1, num_labels_mask+1):
            # Volume and overlap
            cur_mask = t2 == i
            volume = np.sum(cur_mask)
            overlap = np.logical_and(cur_mask, artery)
            overlap_size = overlap.sum()

            x_idx, y_idx, z_idx = np.where(overlap)

            idxx = len(x_idx) // 2
            label = artery_labels[x_idx[idxx], y_idx[idxx], z_idx[idxx]]

            artery_volume = np.sum(artery_labels == label)

            # Aneurysm start and pixel size

            x_s = cur_mask.sum(axis=(1, 2))
            y_s = cur_mask.sum(axis=(0, 2))
            z_s = cur_mask.sum(axis=(0, 1))

            x = np.where(x_s)[0][[0, -1]]
            y = np.where(y_s)[0][[0, -1]]
            z = np.where(z_s)[0][[0, -1]]

            x_start, x_end = x[0], x[1]
            y_start, y_end = y[0], y[1]
            z_start, z_end = z[0], z[1]

            x_size = x_end - x_start
            y_size = y_end - y_start
            z_size = z_end - z_start

            volumes.append((name, overlap_size, artery_volume, volume, x_size, y_size, z_size, x_start, y_start, z_start, x_end, y_end, z_end))


2 / 78: A077.h5
3 / 78: A089_R.h5
4 / 78: A093.h5
5 / 78: A046.h5
6 / 78: A105_L.h5
7 / 78: A094_R.h5
8 / 78: A024.h5
9 / 78: A108.h5
10 / 78: A113.h5
11 / 78: A119.h5
12 / 78: A060.h5
13 / 78: A038_L.h5
14 / 78: A032.h5
15 / 78: A078_L.h5
16 / 78: A134.h5
17 / 78: A026.h5
18 / 78: A010.h5
19 / 78: A096_R.h5
20 / 78: A038_M.h5
21 / 78: A130_R.h5
22 / 78: A092.h5
23 / 78: A080.h5
24 / 78: A095.h5
25 / 78: A112.h5
26 / 78: A096_L.h5
27 / 78: A051_R.h5
28 / 78: A008.h5
29 / 78: A138.h5
30 / 78: A121.h5
31 / 78: A103.h5
32 / 78: A087.h5
33 / 78: A070.h5
34 / 78: A081.h5
35 / 78: A062_L.h5
36 / 78: A028.h5
37 / 78: A027.h5
38 / 78: A015.h5
39 / 78: A079.h5
40 / 78: A043.h5
41 / 78: A064.h5
42 / 78: A126.h5
43 / 78: A066.h5
44 / 78: A085.h5
45 / 78: A084.h5
46 / 78: A044.h5
47 / 78: A074.h5
48 / 78: A003.h5
49 / 78: A040.h5
50 / 78: A133.h5
51 / 78: A017_L.h5
52 / 78: PA6.h5
53 / 78: A014.h5
54 / 78: A012.h5
55 / 78: A100.h5
56 / 78: A056.h5
57 / 78: A001.h5
58 / 78: A123.h5
59 / 78: A057.h5

In [35]:
import pandas as pd
df = pd.DataFrame(volumes, columns=["file", "overlap_size", "artery_volume", "volume", "x_size", "y_size", "z_size", "x_start", "y_start", "z_start", "x_end", "y_end", "z_end"])
df["overlap_to_volume_ratio"] = df["overlap_size"] / df["volume"]

# df = pd.read_csv("/media/lm/Samsung_T5/Uni/Medml/training/train/h5_zoomed/stats.csv")

df.describe()
# df.sort_values(by="x_size", ascending=True).head(10)

# df.to_csv("/media/lm/Samsung_T5/Uni/Medml/training/train/h5_zoomed/stats.csv")


,overlap_size,artery_volume,volume,x_size,y_size,z_size,x_start,y_start,z_start,x_end,y_end,z_end,overlap_to_volume_ratio
count,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000,87.000000
mean,312.126437,8994.942529,352.137931,6.540230,6.482759,6.701149,62.517241,45.609195,40.666667,69.057471,52.091954,47.367816,0.870778
std,646.625081,2615.956201,713.402952,3.747261,3.821184,4.324390,16.299424,11.309963,12.803221,16.068427,11.926746,12.422892,0.076145
min,9.000000,2253.000000,12.000000,1.000000,2.000000,1.000000,25.000000,14.000000,12.000000,29.000000,16.000000,21.000000,0.641638
25%,48.500000,7632.000000,60.000000,4.000000,4.000000,4.000000,52.000000,39.500000,33.000000,61.000000,46.000000,40.000000,0.822128
50%,105.000000,9013.000000,122.000000,5.000000,6.000000,6.000000,64.000000,44.000000,40.000000,69.000000,52.000000,46.000000,0.883534
75%,293.000000,10453.000000,328.500000,8.500000,8.000000,8.000000,71.000000,51.500000,48.000000,78.000000,58.000000,54.000000,0.920115
max,4595.000000,16783.000000,5006.000000,20.000000,20.000000,26.000000,94.000000,84.000000,77.000000,100.000000,89.000000,80.000000,1.000000


In [10]:
# fit mask to artery

datapath = "/media/lm/Samsung_T5/Uni/Medml/training/train/h5"

files = os.listdir(datapath)
sorted(files)

volumes = []
for idx, name in enumerate(files):
    if not name.endswith(".h5"):
        continue

    print(f"{idx} / {len(files)}: {name}")

    with h5py.File(os.path.join(datapath, name), "r+") as f:
        raw = f["raw"][:]
        mask = f["label"][:]
        artery = f["artery"][:]

        artery = ndi.binary_closing(artery, iterations=3)

        overlap_mask = np.logical_and(mask, artery)

        f.create_dataset("overlap_mask", data=overlap_mask)





0 / 76: A077.h5
1 / 76: A089_R.h5
2 / 76: A093.h5
3 / 76: A046.h5
4 / 76: A105_L.h5
5 / 76: A094_R.h5
6 / 76: A024.h5
7 / 76: A108.h5
8 / 76: A113.h5
9 / 76: A119.h5
10 / 76: A060.h5


KeyboardInterrupt: 